In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
bwght = pd.read_csv('bwght.csv')

In [3]:
import csv
import numpy as np
import scipy.stats as sps
from scipy.stats import norm
import pandas as pd


def cross(*args):  # Can put either 1 or 2 things into this function
    x = args[0]
    if len(args) == 1:
        y = x
    else:
        y = args[1]
    return (x.T.dot(y))


ls = lambda X, y: np.linalg.solve(cross(X), cross(X, y))


class ols:
    def __init__(self, X, y):
        self.n = X.shape[0]
        print(X.shape)
        self.X = np.hstack((np.ones((self.n, 1)), X))
        self.y = y
        self.b = ls(self.X, self.y)
        self.pred = self.X.dot(self.b)
        self.res = self.y - self.pred
        RSS = cross(self.res)[0, 0]
        ybar = self.y.mean()
        e_0 = self.y - ybar
        TSS = cross(e_0)[0, 0]
        self.rsq = 1 - RSS / TSS
        sig = np.sqrt(RSS / self.n)
        self.vcov = np.linalg.inv(cross(self.X)) * (sig ** 2)
        
        self.logLikelihood = -self.n/2 * np.log(2*np.pi*sig**2) - self.n/(2*sig**2) * self.res.T.dot(self.res)/self.n
        self.logLikelihood = self.logLikelihood[0,0]
        print("log likelihood", self.logLikelihood)
    def predict(self, newX):
        newn = newX.shape[0]
        newX = np.hstack((np.ones((newn, 1)), newX))
        return newX.dot(self.b)

    def tidy(self):
        se = np.sqrt(np.diag(self.vcov)).reshape(-1, 1)
        tstats = self.b / se
        pval = norm.cdf(-np.abs(tstats)) * 2
        outp = np.hstack((self.b, se, tstats, pval))
        outp = pd.DataFrame(outp)
        outp.columns = ['beta', 'stderr', 'tstat', 'pval']
        print(outp)
        print("RSq=" + str(self.rsq))
        
        
ols(bwght[['cigs','faminc']].values, bwght['bwght'].values.reshape((-1,1)))

(1388, 2)
log likelihood -6130.414104234555


In [6]:
Xdata = pd.concat((bwght['cigs'],bwght['faminc']),axis=1)
Xdata = sm.add_constant(Xdata)
model1 = sm.OLS(bwght['bwght'],Xdata).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  bwght   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     21.27
Date:                Fri, 05 Oct 2018   Prob (F-statistic):           7.94e-10
Time:                        21:52:59   Log-Likelihood:                -6130.4
No. Observations:                1388   AIC:                         1.227e+04
Df Residuals:                    1385   BIC:                         1.228e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        116.9741      1.049    111.512      0.000     114.916     119.032
cigs          -0.4634      0.092     -5.060      0.000      -0.643      -0.284
faminc         0.0928      0.029      3.178      0.002       0.036       0.150
==============================================================================
Omnibus:                      116.751   Durbin-Watson:                   1.922
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              619.781
Skew:                          -0.154   Prob(JB):                    2.61e-135
Kurtosis:                       6.259   Cond. No.                         67.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""